In [2]:
import datetime, os, sys, pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
from lambeq import NumpyModel, AtomicType, IQPAnsatz
from lambeq.backend.quantum import Ry, Diagram, Bra
from util import data_loader
import tensornetwork as tn
from scipy.linalg import logm, sqrtm
from contextuality.model import CyclicScenario

In [3]:
# Measurement basis used in max violation CHSH experiment with their matrix representations
bases = {'a':Ry(0), 'A':Ry(np.pi/4), 'b':Ry(np.pi/8), 'B':Ry(3*np.pi/8)}
contexts = {'ab': np.kron(Ry(0).array, Ry(np.pi/8).array),
            'aB': np.kron(Ry(0).array, Ry(3*np.pi/8).array),
            'Ab': np.kron(Ry(np.pi/4).array, Ry(np.pi/8).array),
            'AB': np.kron(Ry(np.pi/4).array, Ry(3*np.pi/8).array)}

In [4]:
class QModel(NumpyModel):
    def __init__(self, use_jit: bool = False) -> None:
        super().__init__(use_jit)

    def get_output_state(self, diagrams):
        diagrams = self._fast_subs(diagrams, self.weights)
        results = []
        for d in diagrams:
            assert isinstance(d, Diagram)
            result = tn.contractors.auto(*d.to_tn()).tensor
            result = np.array(result).flatten()
            result = np.sqrt(result/sum(abs(result)))
            results.append(result)
        return np.array(results)

In [5]:
models = {'disjoint_uncut': 'runs/disjoint_uncut_130E/best_model.lt', 
          'disjoint_cut': 'runs/disjoint_cut_140E/best_model.lt',
          'spider_uncut': 'runs/spider_uncut_200E/best_model.lt', 
          'spider_cut': 'runs/spider_cut_50E/best_model.lt'}

diagrams = {'disjoint_uncut': 'dataset/diagrams/disjoint_uncut.pkl',
            'disjoint_cut': 'dataset/diagrams/disjoint_cut.pkl',
            'spider_uncut': 'dataset/diagrams/spider_uncut.pkl',
            'spider_cut': 'dataset/diagrams/spider_cut.pkl'}

data = {'disjoint_uncut': 'dataset/contextuality_data/scenario442_disjoint_uncut.csv',
        'disjoint_cut': 'dataset/contextuality_data/scenario442_disjoint_cut.csv',
        'spider_uncut': 'dataset/contextuality_data/scenario422_spider_uncut.csv',
        'spider_cut': 'dataset/contextuality_data/scenario442_spider_cut.csv'}

In [9]:
def plot_cnxt(x, y, z, title: str, save: bool=True) -> None:
    cmap = plt.get_cmap('viridis_r')
    cmap.set_under('red')
    scat = plt.scatter(x=x, y=y, c=z, cmap=cmap, vmax=1, vmin=1/6)
    plt.axvline(x=1/6, color='r', linestyle='-')
    plt.axhline(y=2, color='r', linestyle='-')
    plt.text(x=1/6+0.05,y=5,s='Sheaf Contextual')
    plt.text(x=0.7,y=1.5,s='CbD Contextual')
    plt.xlabel('Contextual Fraction')
    plt.ylabel('Direct Influence')
    plt.colorbar(label='Signalling Fraction', extend='min')
    plt.title(title)
    scat.set_alpha(0.5)
    scat.cmap.set_over('red')
    if save:
        plt.savefig('figures/' + title + '_' + datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S"))
    plt.show()

In [10]:
def calc_violation(state: np.array) -> float:
    expecations = [np.conjugate(state) @ (contexts[ops] @ state) for ops in list(contexts.keys())]
    return max([sum(expectations) - 2*exp for exp in expectations])

In [19]:
def density_op(state, tol=1e-12):
    dense_mat = np.outer(state, np.conjugate(state))
    dense_mat.real[abs(dense_mat.real) < tol] = 0.0
    dense_mat.imag[abs(dense_mat.imag) < tol] = 0.0
    return dense_mat

In [93]:
def log_mat(mat): # Matrix logarithm via eigendecomposition
    evals, emat = np.linalg.eig(mat) # Get matrix V of eigenvectors of input matrix A
    emat_inv = np.linalg.inv(emat) # Get inverse of matrix V
    matp = emat @ mat @ emat_inv # Compute A' with a diagonal of eigenvalues tr(A') = evals
    tr = matp.diagonal() # Get the trace of the matrix
    np.fill_diagonal(matp, np.log2(tr, out=np.zeros_like(tr, dtype=np.complex128), where=(tr!=0))) # Element wise base 2 log of diagonal
    # Line above ignores log(0) error by replacing it with 0, which may lead to a wrong answer
    return emat_inv @ matp @ emat # Change basis back

In [44]:
def partial_trace(dense_mat):
    # Compute reduced density matrix for a bipartide quantum system
    dims_a = int(2**(np.floor(np.log2(dense_mat.shape[0])/2)))
    dims_b = int(2**(np.ceil(np.log2(dense_mat.shape[0])/2)))
    id_a = np.identity(dims_a)
    id_b = np.identity(dims_b)

    rho_a = np.zeros((dims_a, dims_a))
    rho_b = np.zeros((dims_b, dims_b))

    for base in id_b:
        bra = np.kron(id_a, base)
        ket = np.kron(id_a, base).T
        rho_a = rho_a + (bra @ dense_mat) @ ket
    for base in id_a:
        bra = np.kron(id_b, base)
        ket = np.kron(id_b, base).T
        rho_b = rho_b + (bra @ dense_mat) @ ket        
    return rho_a, rho_b

In [118]:
def calc_vne(dense_mat, direct=True):
    if direct:
        return -np.trace(dense_mat @ log_mat(dense_mat))
    evals = np.linalg.eigvals(dense_mat)
    evals = evals[np.abs(evals) > 1e-12]
    ent = -np.sum(evals * np.log2(evals))
    if ent > 1e-12:
        return ent
    else:
        return 0

In [18]:
def qrel_ent(mat1, mat2):
    evals1 = self.eigen_decompose(mat1)
    evals2 = self.eigen_decompose(mat2)
    return np.sum(evals1*(evals1 - evals2))

In [339]:
data = data_loader(scenario=CyclicScenario(['a','b','A','B'],2),
                      model_path=models['spider_cut'])

In [340]:
data.get_data(data['spider_cut'])
data.get_diagrams(diagrams['spider_cut'])

In [341]:
x = test.data['CF']
y = test.data['Violation']
z = test.data['SF']
plot_cnxt(x, y, z, "Hello")

In [132]:
eoe = []
qre = []
for diagram in tqdm(test.diagrams):
    try:
        #dist = test.model.get_diagram_output([diagram])[0].flatten()
        #state = np.sqrt(dist)
        state = test.model.get_output_state([diagram])[0]
        dense_mat = test.get_dense_mat(state)
        reduced_dense_mat = test.bipartide_partial_trace(dense_mat)
        eoe.append(test.calc_vne(reduced_dense_mat))

        diag1 = diagram.apply_gate(Bra(0),0)
        diag2 = diagram.apply_gate(Bra(0),1)
        state1 = test.model.get_output_state([diag1])[0]
        state2 = test.model.get_output_state([diag2])[0]
        mat1 = test.get_dense_mat(state1)
        mat2 = test.get_dense_mat(state2)
        qre.append(test.qrel_ent(mat1, mat2))
    except Exception as err:
        tqdm.write(f"Error: {err}".strip(), file=sys.stderr)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3057/3057 [03:38<00:00, 14.02it/s]


In [121]:
state = np.array([1,0,0,0,0,0,0,1],dtype=np.cdouble)*np.sqrt(1/2)
#state = np.array([1,0,0,1],dtype=np.cdouble)*np.sqrt(1/2)
dense_mat = density_op(state)
reduced_a, reduced_b = partial_trace(dense_mat)
calc_vne(reduced_a)

(0.9999999999999999-0j)